In [77]:
import matplotlib
matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os
import umap
import parmap

import glob2
import numpy as np
from sklearn.decomposition import PCA

# visualize results module
from Visualize import Visualize
from Visualize import PCA_Analysis
from Visualize import *

import pickle as pk
from sklearn.linear_model import LinearRegression

# 
#data_dir = '/media/cat/4TBSSD/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [2]:
##################################################
######### PROCESS CONVEX HULL FOR ALL DATA #######
##################################################

# ************** FIG 2E Convex hull analysis



root_dir = '/media/cat/4TBSSD/yuki/'

animal_ids = ['IA1','IA2','IA3','IJ1',"IJ2","AQ2"]
session_id = 'all'

# 
for animal_id in animal_ids:
    pa = PCA_Analysis()
    pa.root_dir = '/media/cat/4TBSSD/yuki/'
    pa.session_id = 'all'
    pa.animal_id = animal_id

    # 
    pa.n_pca = 10
    pa.sliding_window = 30    # how many frames to take into analysis window
    pa.n_frames = 300         # how many frames back in time to analyze: 

    
    # 
    sessions = get_sessions(pa.root_dir,
                            pa.animal_id,
                            pa.session_id)


    for session in sessions:
        pa.session = session
        print (" processing: ", session)
        fname_out = os.path.join(pa.root_dir,
                                  pa.animal_id,
                                  'tif_files',
                                  pa.session,
                                  pa.session+'_convex_hull.npz')
        if os.path.exists(fname_out):
            print ("     data laready processed")
            print ('')
            continue
            
        fname_triggers = os.path.join(pa.root_dir,
                                      pa.animal_id,
                                      'tif_files',
                                      pa.session,
                                      'blue_light_frame_triggers.npz')

        fname_data = os.path.join(pa.root_dir,
                                     pa.animal_id,
                                     'tif_files',
                                     pa.session,
                                     pa.session+ '_aligned_butterworth_0.1hz_6.0hz.npy')

        pa.X, pa.triggers = get_data_and_triggers(fname_triggers,
                                                  fname_data)
        
        if pa.X is None:
            print ("     missing data/blue light")
            print ('')
            print ('')
            continue
            
        if pa.triggers.shape[0]<5:
            print ("     not enough points : ", pa.triggers.shape )
            print ('')
            print ('')
            continue
            
        # make PCA plots
        pa.pca, pa.all_points = get_pca_object_and_all_points(pa)


        # FIG 2 E top
        pa.p_lever =  project_data_pca(pa)

        # skip plotting option
        # plot_pca_scatter(pa)
         
        # FIG 2E
        pa =  get_convex_hull(pa)

        # skip plotting option
        # plot_convex_hull(pa)
        print ('')


 processing:  IA1pm_Feb1_30Hz
 triggers:  (3,)
 data:  (40000, 128, 128)
(39130, 128, 128)
X:  (39130, 16384)
     not enough points :  (3,)


 processing:  IA1pm_Feb2_30Hz
     data laready processed

 processing:  IA1pm_Feb3_30Hz
     data laready processed

 processing:  IA1pm_Feb4_30Hz
     missing data/blue light


 processing:  IA1pm_Feb5_30Hz
 triggers:  (4,)
 data:  (40000, 128, 128)
(39080, 128, 128)
X:  (39080, 16384)
     not enough points :  (4,)


 processing:  IA1pm_Feb9_30Hz
     data laready processed

 processing:  IA1pm_Feb10_30Hz
     data laready processed

 processing:  IA1pm_Feb11_30Hz
     data laready processed

 processing:  IA1pm_Feb12_30Hz
 triggers:  (2,)
 data:  (40000, 128, 128)
(39070, 128, 128)
X:  (39070, 16384)
     not enough points :  (2,)


 processing:  IA1pm_Feb15_30Hz
     data laready processed

 processing:  IA1pm_Feb16_30Hz
     data laready processed

 processing:  IA1pm_Feb17_30Hz
     data laready processed

 processing:  IA1pm_Feb18_30Hz
 

 triggers:  (2,)
 data:  (40000, 128, 128)
(39015, 128, 128)
X:  (39015, 16384)
     not enough points :  (2,)


 processing:  AQ2am_Dec22_30Hz
     missing data/blue light


 processing:  AQ2am_Dec23_30Hz
     missing data/blue light


 processing:  AQ2am_Dec28_30Hz
     data laready processed

 processing:  AQ2am_Dec29_30Hz
     data laready processed

 processing:  AQ2am_Dec30_30Hz
     missing data/blue light


 processing:  AQ2am_Dec31_30Hz
     data laready processed

 processing:  AQ2am_Jan4_30Hz
     data laready processed

 processing:  AQ2am_Jan5_30Hz
     data laready processed

 processing:  AQ2am_Jan6_30Hz
     data laready processed

 processing:  AQ2am_Jan7_30Hz
 triggers:  (4,)
 data:  (40000, 128, 128)
(39123, 128, 128)
X:  (39123, 16384)
     not enough points :  (4,)


 processing:  AQ2am_Jan8_30Hz
     data laready processed

 processing:  AQ2pm_Jan11_30Hz
     data laready processed

 processing:  AQ2am_Jan11_30Hz
     missing data/blue light


 processing:  AQ2pm_

In [118]:
########################################################
####### STAND ALONE FUNCTION TO PLOT CONVEX HULL #######
########################################################
animal_ids = ['IA1','IA2','IA3','IJ1',"IJ2","AQ2"]
clrs = ['black','blue','red','green','magenta','pink']

aucs = []
aucs_norm = []

animal_ids = ['AQ2']
for ctr_animal, animal_id in enumerate(animal_ids):
    
    
    aucs.append([])
    aucs_norm.append([])
    
    pa = PCA_Analysis()
    pa.root_dir = '/media/cat/4TBSSD/yuki/'
    pa.session_id = 'all'
    pa.animal_id = animal_id

    # 
    sessions = get_sessions(pa.root_dir,
                     pa.animal_id,
                     pa.session_id)

    cmap = matplotlib.cm.get_cmap('Reds')

    clr_ctr=0
    n_sessions = len(sessions)
    alpha = .5

    fig=plt.figure(figsize=(7,5))
    for session in sessions:
        # 
        fname = os.path.join(pa.root_dir, pa.animal_id,'tif_files',session,
                            session+ '_convex_hull.npz')
        # fname = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb2_30Hz/IA1pm_Feb2_30Hz_convex_hull.npz'
        try:
            d = np.load(fname, allow_pickle = True)
        except:
            clr_ctr+=1
            continue

        pa.ratio_single = d['ratio_single'][::-1]
        pa.ratio_cumsum = d['ratio_cumsum'][::-1]
        pa.ratio_random_single = d['ratio_random_single'][::-1]
        pa.ratio_random_cumulative = d['ratio_random_cumulative'][::-1]

        # 
        plot_convex_hull_cumulative_only(pa,
                                         cmap,
                                         clr_ctr/n_sessions,
                                         alpha)


        # 
        #plot_convex_hull_single_only(pa,
        #                            cmap,
        #                            clr_ctr/n_sessions,
        #                            alpha)
        clr_ctr+=1


        aucs[ctr_animal].append(pa.ratio_cumsum.sum())
        aucs_norm[ctr_animal].append((pa.ratio_cumsum/np.max(pa.ratio_cumsum)).sum())

    
if True:
    plt.savefig('/home/cat/convex.png',dpi=300)
    plt.close()
else:
    plt.show()

In [116]:
###########################################
################# AUC STATS ###############
###########################################
# plot AUC

def find_p_t(model, X, y, X2, y_pred):
    #self = super(LinearRegression, self).fit(X, y, n_jobs)
    #print (self)

    #print (y_pred.shape, y.shape)
    sse = np.sum((y_pred - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
    
    #print (sse)
    se = np.array([
        np.sqrt(np.diagonal(sse[i] * np.linalg.inv(np.dot(X.T, X))))
                                                for i in range(sse.shape[0])
                ])

    t = model.coef_ / se
    p = 2 * (1 - stats.t.cdf(np.abs(t), y.shape[0] - X.shape[1]))
    
    return t, p

# 
clrs = ['black','blue','red','green','magenta','pink']
animal_ids = ['M1','M2','M3','M4','M5','M6']

sig = [0.05,0.01,0.001, 0.0001, 0.00001]
fig = plt.figure(figsize=(6,6))
ax=plt.subplot(111)
for k in range(len(aucs)):
    
    temp = aucs_norm[k]
    plt.scatter(np.arange(len(temp)), temp,
                c=clrs[k],
                edgecolor='black',
                s=100,
                alpha=.5)

    # fit
    model = LinearRegression()
    y=np.array(temp).reshape(-1, 1)
    x = np.arange(y.shape[0]).reshape(-1, 1)
    model.fit(x, y)

    x2 = np.arange(0,y.shape[0],1).reshape(-1, 1)
    y_pred = model.intercept_ + model.coef_ * x2


    t,p = find_p_t(model, x, y, x2, y_pred)
    
    temp_sig = ''
    for j in range(len(sig)):
        if p <= sig[j]:
            temp_sig = temp_sig+"*"
            
    
    plt.plot(x2, y_pred, label=animal_ids[k]+ " "+temp_sig,
             c=clrs[k],
            linewidth=6)
    print (t)
    print (p)
    print('')
    
plt.legend(fontsize=12)
plt.xlim(x[0],x[-1])
plt.ylim(np.min(y), np.max(y))
plt.xticks([])
plt.yticks([])

plt.savefig('/home/cat/allan.png', dpi=300)
plt.close()

[[10.73731016]]
[[2.66453526e-15]]

[[-0.62458662]]
[[0.53653721]]

[[3.17776025]]
[[0.00282089]]

[[-1.44728581]]
[[0.15560928]]

[[3.45494834]]
[[0.00136843]]

[[9.84406994]]
[[1.55431223e-15]]



In [123]:
fig=plt.figure()
ax=plt.subplot(111)
data = np.arange(100, 0, -1).reshape(10, 10)

im = ax.imshow(data, cmap='jet')

fig.colorbar(im)
plt.show()
    

In [4]:
#############################################
##### COMPUTE PCA ON EACH SELECTED VALUE ####
#############################################

# FIG 2 E top
pa.p_lever =  project_data_pca(pa)
 
# 
plot_pca_scatter(pa)



100%|██████████| 300/300 [01:18<00:00,  3.85it/s]

 pca p_levefr resahped:  (300, 44, 10)


In [7]:
#####################################################
####### COMPUTE AND PLOT CONVEX HULL DYNAMICS #######
#####################################################

# FIG 2E
#
pa =  get_convex_hull(pa)

#  
plot_convex_hull(pa)

  7%|▋         | 20/300 [00:00<00:01, 190.74it/s]

temp points:  (1314, 3)
# points kept:  (1312,)  of total:  1314
computing convex hull # dim:  3 (1312, 3)
p lever:  (300, 44, 10)


100%|██████████| 300/300 [00:07<00:00, 38.67it/s]


(300, 10)
(300,)


In [64]:
################################################################
################################################################
################################################################

# plot convex hull results


In [59]:
# # UMAP ON DATA
X_U = []
X_U.append(X_lever)
X_U.append(X_random)
X_U.append(X_shifted)
X_U = np.vstack(X_U)
print ("XU_:", X_U.shape)

import umap
umap_3d = umap.UMAP(n_components=3, init='random', random_state=0)

#fit = umap.UMAP()
%time umap_3d.fit(X_U[::10])
u = umap_3d.transform(X_U)

print ("Umap output: ", u.shape)


XU_: (339, 491520)
CPU times: user 1min 6s, sys: 1.09 s, total: 1min 7s
Wall time: 1min 7s
Umap output:  (339, 3)


In [61]:
fig=plt.figure()
#ax = plt.subplot(111)
ax = fig.add_subplot(projection='3d')
ax.scatter(
           u[:p_lever.shape[0],0], 
           u[:p_lever.shape[0],1],
           u[:p_lever.shape[0],2],
            c='red',
            s=100,
            edgecolor = 'black', alpha=1)

ax.scatter(
           u[p_lever.shape[0]:p_lever.shape[0]+p_random.shape[0],0], 
           u[p_lever.shape[0]:p_lever.shape[0]+p_random.shape[0],1], 
           u[p_lever.shape[0]:p_lever.shape[0]+p_random.shape[0],2], 
            c='black',
            s=100,
            edgecolor = 'black', alpha=1)

ax.scatter(
           u[p_lever.shape[0]+p_random.shape[0]:,0], 
           u[p_lever.shape[0]+p_random.shape[0]:,1], 
           u[p_lever.shape[0]+p_random.shape[0]:,2], 
    
            c='pink',
            s=100,
            edgecolor = 'black', alpha=1)
plt.show()

In [69]:
############################################
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

#ax.scatter(xs, ys, zs, marker=m)
ax.scatter(p[:,0], p[:,1], p[:,2],
            c=np.arange(p.shape[0]), 
            s=10, cmap=plt.cm.viridis,
            edgecolor = 'black', alpha=1)
    
ax.scatter(p_lever[:,0], p_lever[:,1],  p_lever[:,2], 
            c='red',
            s=100,
            edgecolor = 'black', alpha=1)

ax.scatter(p_random[:,0], p_random[:,1], p_random[:,2], 
            c='pink',
            s=100,
            edgecolor = 'black', alpha=1)





plt.show()  

In [7]:
############################################
############################################
############################################
fig=plt.figure()
ax=plt.subplot(221)
plt.scatter(p[:,0], p[:,1], 
            c=np.arange(p.shape[0]), 
            s=10, cmap=plt.cm.viridis,
            edgecolor = 'black', alpha=1)
    
plt.scatter(p[triggers,0], p[triggers,1], 
            c='red',
            s=100,
            edgecolor = 'black', alpha=1)
    

ax=plt.subplot(222)
plt.scatter(p2[:,0], p2[:,1], 
            c=np.arange(p2.shape[0]), 
            s=10, cmap=plt.cm.viridis,
            edgecolor = 'black', alpha=1)
    
plt.scatter(p2[triggers,0], p2[triggers,1], 
            c='red',
            s=100,
            edgecolor = 'black', alpha=1)
    
    
ax=plt.subplot(223)
plt.scatter(u[:,0], u[:,1], 
            c=np.arange(u.shape[0]), 
            s=10, cmap=plt.cm.viridis,
            edgecolor = 'black', alpha=1)
    
plt.scatter(u[triggers,0], u[triggers,1], 
            c='red',
            s=100,
            edgecolor = 'black', alpha=1)

plt.show()

In [53]:
#############################################################
#### PLOT DISTRIBUTION OF FIRST SIG DECODING ALL ANIMALS ####
#############################################################

# return all session ids that can be predicted longer than this:
return_ids_threshold = -10

vis.window = 30
vis.lockout_window = 10

# 
vis.min_trials = 20  #min trials used for SVM

vis.lockout=False
# 
vis.plot_first_decoding_time(return_ids_threshold)

# 
if True:
    plt.ylim(-15,0)
    plt.show()
else:
    plt.savefig('/home/cat/first.png',dpi=600)
    plt.close()

all:  ['AQ2am_Feb12_30Hz' 'AQ2am_Feb15_30Hz']
all:  [60 61]
[[-2.8000000000000007, -7.266666666666666, -3.7333333333333343, -2.366666666666667, -3.033333333333335, -3.1999999999999993, -4.033333333333335, -3.466666666666665, -2.2666666666666657, -1.2333333333333343, -3.666666666666668, -2.533333333333335, -2.866666666666667, -2.966666666666665, -2.0666666666666664, -1.0, -2.0666666666666664, -3.2333333333333343, -2.333333333333332, -2.333333333333332, -1.9333333333333336, -1.0, -2.2666666666666657, -1.5666666666666664, -2.4333333333333336, -2.0666666666666664, -1.466666666666665, -1.1999999999999993, -2.0666666666666664, -2.0, -1.6999999999999993], [-1.0, -4.300000000000001, -3.7666666666666657, -1.0, -2.966666666666665, -1.0, -2.166666666666668, -1.5, -1.0, -1.0, -2.1000000000000014, -1.0, -1.533333333333335, -1.2666666666666657, -1.0, -1.0, -4.533333333333335, -4.533333333333335, -3.7666666666666657, -4.266666666666666, -3.7333333333333343, -5.300000000000001, -4.133333333333333, -4.

In [184]:
# 
data = np.load('/home/cat/concatenated_trials.npy')
#data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')

fnames = [
'/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb5_30Hz/IA2pm_Feb5_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy',
'/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar2_30Hz/IJ1pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy',
'/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar3_30Hz/IJ2pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy',
'/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Apr1_Week4_30Hz/AQ2am_Apr1_Week4_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy',
]

fig=plt.figure(figsize=(15,10))
for ctr,fname in enumerate(fnames):
    data = np.load(fname)
    print ("raw data: ", data.shape)

    data = data.mean(0)[:,0:900].T

    print (data.shape)

    ax=plt.subplot(2,2,ctr+1)
    #plt.scatter(data[:,0],
    #            data[:,1],
    #            c=np.arange(data.shape[0]))
    t = np.arange(data.shape[0])
    for k in range(4):
        plt.plot(t,data[:,k], linewidth=6, label="PCA "+str(k))
    plt.xticks([])
    plt.yticks([])
    if ctr==0:
        plt.legend(fontsize=14)
    plt.xlim(t[0],t[-1])
    plt.ylim(-4,4)
    
if True:
    plt.savefig('/home/cat/pca_vs_time.png',dpi=300)
    plt.close()
else:
    plt.show()


raw data:  (67, 5, 1801)
(900, 5)
raw data:  (61, 7, 1801)
(900, 7)
raw data:  (48, 6, 1801)
(900, 6)
raw data:  (137, 6, 1801)
(900, 6)


In [1]:
from scipy.optimize import curve_fit

def func(x, a, b, c, d):
    return a*np.exp(-c*(x-b))+d

fig=plt.figure()        
plt.scatter(vis.predictions1,
    vis.trials1)
plt.ylim(0,160)
plt.xlim(-10,0)


# fit exponentials
popt, pcov = curve_fit(func,
                       vis.predictions1+30,
                       vis.trials1,
                       [11,1e-6,1e-3,3],
                       maxfev=10000)

print("Popt: ", popt)

x= np.linspace(0,10,1000)
plt.plot(x-10,func(x,*popt))
            
        
print (vis.trials1)
print (vis.predictions1)

NameError: name 'plt' is not defined

In [5]:
#################################################
######### COMPARE RNN VS SVM DECODING ###########
#################################################
# 
fnames_svm = glob2.glob('/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/*.npy')
fnames_rnn= glob2.glob('/media/cat/4TBSSD/yuki/IJ2/RNN_scores/*.npz')

vis.compare_svm_rnn(fnames_svm, fnames_rnn)



In [8]:
#######################################
######## PLOT SVM DECISION TIME #######
#######################################
svm = PredictSVMTime()
svm.main_dir = '/media/cat/4TBSSD/yuki/'

# default params
svm.lockout = False
svm.lockout_window = 10
svm.pca_flag = True
svm.pca_var = 0.95
svm.window = 10

fig = plt.figure()
svm.animal_id = 'AQ2'
svm.session_id = 'all'
svm.get_sessions()

#
ctr=1
for session_id in svm.sessions:
    ax=plt.subplot(8,10,ctr)
    svm.session_id = session_id
    try:
        svm.plot_decision_time(ax)
        plt.title(session_id+", #: "+str(svm.n_trials_both[0]), fontsize=6)
        if ctr!=1:
            plt.yticks([])
        if ctr!=31:
            plt.xticks([])

        ctr+=1

    except:
        pass
plt.suptitle(svm.animal_id,fontsize=20)



Text(0.5, 0.98, 'AQ2')

In [4]:
data = np.load('/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2am_Mar7_30Hz_code_04_trial_ROItimeCourses_10sec_pca_0.95.npy')
print (data.shape)

(601, 10)


In [ ]:

# vis.main_dir = '/media/cat/4TBSSD/yuki/output_3.7-20210310T074752Z-001/output_3.7/no_midline_filter/'

# fname = vis.main_dir+'/'+animal_id+'/SVM_scores_'+animal_id+'_'+str(session)+'.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('black','all - 30frame windows')

# fname = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/analysis/all_accuracy.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('red','all - 1 frame windows')

# fname = vis.main_dir+'/'+animal_id+'/SVM_scores_'+animal_id+'_lockout_'+str(session)+'.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('magenta','lockout')

# # rnn
# fname = vis.main_dir+'/'+animal_id+'/acc_rnn_'+animal_id+'_'+str(session)+'.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('green','rnn-all-denoised')

# fname = vis.main_dir+'/'+animal_id+'/acc_rnn_'+animal_id+'_lockout_'+str(session)+'.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('cyan','rnn-lockout-denoised')


# # #
# # vis.load_data(fname)
# fname = '/media/cat/4TBSSD/yuki/march_7/IA1pm_Feb23_30Hz_code_04_PCA_Denoised/analysis/all_accuracy.npy'
# #fname = vis.main_dir+'/'+animal_id+'/SVM_scores_'+animal_id+'_'+str(session)+'.npy'

# vis.load_data(fname)
# vis.plot_significant('red','all',animal_id, session)
 
# #
# vis.plot_animal_decision_longitudinal('IA1')

# # 
# vis.plot_animal_decision_AUC_longitudinal()

# # # # # 
# lockout=False
# for ctr, name in enumerate(vis.animal_names):
#     ax=plt.subplot(2,3,ctr+1)
#     vis.plot_animal_decision_longitudinal_matrix(name, lockout, ax)
    

#############################################
############## DECISION TIME ################
#############################################

#
# vis.main_dir = '/media/cat/1TB/yuki/yongxu/lever pull/'

# fname = '/media/cat/4TBSSD/yuki/output_3.7-20210310T074752Z-001/output_3.7/no_midline_filter/IA1/conf_10_IA1_0.npy'
# vis.load_data(fname)
# vis.plot_decision_time('red','all')

# animal_name = "IA1"
# vis.plot_decision_time_animal(animal_name)

# animal_name = 'IA1'
# vis.plot_decision_time_animal_matrix(animal_name)

# # 
# vis.filter=False
# vis.plot_decision_time_all_matrix()



#############################################
############## BODY MOVEMENTS ###############
#############################################

# body movement related data
#vis.main_dir = '/media/cat/1TB/yuki/yongxu/body movement/'

# # 
# body_part = 'right_paw'
# fname = vis.main_dir+'/SVM_scores_'+body_part+'_1.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('red','1', body_part)

# fname = vis.main_dir+'/SVM_scores_'+body_part+'_2.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('blue','2', body_part)

# # # 
# fname = vis.main_dir+'/conf_10_left_paw.npy'
# vis.load_data(fname)
# vis.plot_decision_time('red','left')

# fname = vis.main_dir+'/conf_10_right_paw.npy'
# vis.load_data(fname)
# vis.plot_decision_time('blue','right')

# fname = vis.main_dir+'/conf_10_tongue.npy'
# vis.load_data(fname)
# vis.plot_decision_time('green','tongue')


In [7]:
import pickle as pk

file = open('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_10sec_pca.pkl', 'rb')

# dump information to that file
pca = pk.load(file)

print (pca.components_.shape)


(1500, 16384)


In [9]:
data_stm = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_10.0sec_butterworth_globalAverage_0.1hz_6.0hz_04code_stm.npy')
print (data_stm.shape)

(62, 601, 128, 128)


In [10]:
# 
X = data_stm.reshape(data_stm.shape[0]*data_stm.shape[1],
                     data_stm.shape[2]*data_stm.shape[3])

mu= np.mean(X, axis=0)


NameError: name 'nComp' is not defined

In [11]:
nComp = 9
Xnew = np.dot(pca.transform(X)[:,:nComp],
             pca.components_[:nComp,:])

Xnew+=mu

data_stm_denoised_reshaped = Xnew.reshape(data_stm.shape[0], data_stm.shape[1],
                                          data_stm.shape[2], data_stm.shape[3])


In [17]:
time_filters = pca.transform(X)[:,:nComp]
pca_time_filters_only = time_filters.reshape(data_stm.shape[0], data_stm.shape[1],-1).transpose(0,2,1)
print (pca_time_filters_only.shape)


(62, 9, 601)


(62, 601, 9)


In [16]:
data2 = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_10sec.npy')
print (data2.shape)

(40, 35, 601)


In [12]:
print (Xnew.shape)

(37262, 16384)


In [13]:
data_stm_denoised_reshaped.shape

(62, 601, 128, 128)

In [6]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files.npy')
print (data)

['/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb2_30Hz/IA1pm_Feb2_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb4_30Hz/IA1pm_Feb4_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb5_30Hz/IA1pm_Feb5_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb9_30Hz/IA1pm_Feb9_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb10_30Hz/IA1pm_Feb10_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb11_30Hz/IA1pm_Feb11_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb12_30Hz/IA1pm_Feb12_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb15_30Hz/IA1pm_Feb15_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb16_30Hz/IA1pm_Feb16_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_f

In [7]:
data = np.load('/media/cat/1TB/yuki/IA1/tif_files/IA1pm_Feb23_30Hz/IA1pm_Feb23_30Hz_code_04_trial_ROItimeCourses_10sec_pca_0.5.npy')
print (data.shape)

(82, 2, 601)


In [9]:
# #
# data1 = np.load('/home/cat/acc_rnn.npy.npz')
# b = data1['b_rnn']
# c = data1['c_s']

# #
# print (b.shape)
# mean = b.mean(1)
# t=np.linspace(-9.5, 9.5, mean.shape[0])
# plt.plot(t, mean, c='black')
# plt.fill_between(t, mean+c, mean-c, color='black', alpha=.2)

# # 
# data2 = np.load('/home/cat/acc_rnn.npy.npz')
# b = data2['b_rnn']
# c = data2['c_s']
# mean = b.mean(1)
# plt.plot(t, mean, 'blue')
# plt.fill_between(t, mean+c, mean-c, color='blue', alpha=.2)


# plt.show()

In [13]:
import numpy as np
import matplotlib.pyplot as plt
data= np.load('/media/cat/1TB/yuki/maskwarp.npy')
print (data.shape)

plt.imshow(data)
plt.show()

(128, 128)


In [138]:
# 
def sum_pixels_in_registered_mask(data, 
                                  maskwarp,
                                  min_pixels=10):

    
    # 
    areas = np.unique(maskwarp)

    
    
    # work in 1D vectors easier to mask
    maskwarp1D = maskwarp.reshape(-1)
    #maskwarp1D = maskwarp

    trial_courses = []
    area_ids = []
    for k in range(data.shape[0]):
        time_courses_local = []

        # convert to 1D vector to mask faster
        data1D = np.float32(data[k].reshape(data[k].shape[0],-1))

        # 
        print (k,"Data1D: ", data1D.shape)
        for id_ in areas:
            idx = np.where(maskwarp1D==id_)[0]
            #idx = np.where(maskwarp1D==id_)

            # only keep areas that have at least 10 pixels
            if idx.shape[0]>min_pixels:
                area_ids.append(id_)
                
                temp = data1D[:,idx]

                if False:
                    # compute DFF
                    F0 = np.nanmean(temp,axis=0)
                    dFF = (data1D[:,idx]-F0)/F0
                else:
                    # skip dFF computation;
                    dFF = temp

                # save average of all pixesl post DFF
                time_courses_local.append(np.nanmean(dFF, axis=1))

            #all_times.append(time_courses_local)
        trial_courses.append(time_courses_local)

    area_ids = np.int32(np.unique(area_ids))
    trial_courses = np.float32(trial_courses)
    print ("# trials, # areas, # times: ", trial_courses.shape)
    print ("area ids: ", area_ids.shape)

    return area_ids, trial_courses

# 
def check_neighbours(mask_roi, idx, p):
    
    x = idx[0][p]
    y = idx[1][p]
    
    sums = 0
    for k in range(-1,2,1):
        for p in range(-1,2,1):
            xx = x+k
            yy = y+p
            if xx < 0 or xx> 127 or yy<0 or yy>127:
                pass
            else:
                sums+=mask_roi[xx,yy]
                if sums>=2:
                    return True
    return False

# 
def remove_single_pixels(maskwarp,
                         min_pixels=10):

    
    
    # load brain mask (i.e. no tissue areas) and apply it to the image mask first
    if True:
        temp = np.int32(np.loadtxt('/media/cat/1TB/yuki/IA1/genericmask.txt'))
        brain_mask = np.ones((128,128),'float32')
        for t in temp:
            brain_mask[t[0],t[1]]=0

        maskwarp = maskwarp*brain_mask
    
    # 
    ids = np.unique(maskwarp)

    ids_selected = []
    masks = []
    mask_b = np.zeros((128,128),'int32')
    for id_ in ids: 
        idx = np.where(maskwarp==id_)
        if idx[0].shape[0]>min_pixels:
            
            # make the mask for the specific ROI
            mask_roi = np.zeros((128,128))
            idx = np.where(maskwarp==id_)

            mask_roi[idx] = 1
            
            #idx_connected = []
            x = []
            y = []
            for p in range(idx[0].shape[0]):
                connected = check_neighbours(mask_roi, idx, p)
                if connected:
                    x.append(idx[0][p])
                    y.append(idx[1][p])
            
            # 
            if len(x)>min_pixels:
                ids_selected.append(int(id_))
                temp = mask_b.copy()
                for p in range(len(x)):
                     temp[x[p],y[p]]=1
                
                masks.append(temp)
                
        
    ids_selected = np.array(ids_selected)
    return ids_selected, masks
# 
maskwarp= np.load('/media/cat/1TB/yuki/maskwarp.npy')

min_pixels = 50
ids, masks = remove_single_pixels(maskwarp,
                                 min_pixels)
print (ids, len(ids))

#ids, courses = sum_pixels_in_registered_mask(data, maskwarp)

# remove certain specific areas:
idx_del =  [0,16,20]
ids = np.delete(ids,0,0)
masks = np.delete(masks,0,0)
print (ids, len(ids))


[  0  15  21  29  36  43  50  57  64  71 136 150 164 186 198 249 255 261
 268 275] 20
[ 15  21  29  36  43  50  57  64  71 136 150 164 186 198 249 255 261 268
 275] 19


In [141]:
# Load names of ROIs
with open('/home/cat/dorsalMaps_name.txt') as f:
    fs = list(f)

fs = np.vstack(fs)
names = []
for name in fs:
    temp = name[0][1:-2]
    names.append(temp)

# load brain mask (i.e. no tissue areas)
temp = np.int32(np.loadtxt('/media/cat/1TB/yuki/IA1/genericmask.txt'))
mask = np.ones((128,128),'float32')
for t in temp:
    mask[t[0],t[1]]=np.nan
    
    
# Plot ROIs
rois = []
final_names = []
for ctr, id_ in enumerate(ids):
    ax=plt.subplot(5,7,ctr+1)
    temp = np.zeros((128,128))+np.nan

    roi = masks[ctr]*mask
    plt.imshow(roi)
    rois.append(roi)
    #
    plt.xticks([])
    plt.yticks([])
    plt.ylabel(str(id_))
    plt.title(names[id_], fontsize=6)
    final_names.append(names[id_])
    
plt.show()
np.savez('/home/cat/rois_'+str(min_pixels)+'.npz',
        rois = rois,
        ids = ids,
        names = final_names)

In [120]:
data = np.load('/home/cat/roi.npz')
rois = data['rois']
plt.imshow(rois[1])

In [4]:
data = np.load('/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Apr29_Week8_30Hz/AQ2am_Apr29_Week8_30Hz_code_04_random_ROItimeCourses_15sec_pca_0.95.npy')
print (data.shape)
data2 = np.load('/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Apr29_Week8_30Hz/AQ2am_Apr29_Week8_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95.npy')
print (data2.shape)

(81, 5, 901)
(82, 5, 901)


In [20]:
data = np.load('/media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2am_Dec29_30Hz_code_04_trial_ROItimeCourses_15sec_pca_0.95.npy')
data.shape

(901, 10)

In [29]:
data = np.load('/media/cat/4TBSSD/yuki/first_decoding_time_all.npz', allow_pickle=True)
session_nos = data['all_session_nos']
session_nos


array([list([]), list([]), list([]), list([]), list([]),
       list([15, 17, 18, 19, 20, 27, 28, 29, 30, 32, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109])],
      dtype=object)

In [21]:
# load rnn
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

# 
rnn_dir = '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IJ2/temp/rnn/'
svm_dir = '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/IJ2/temp/svm/'

# session:
session = 'Mar9_'

# 
data_rnn = np.load(glob.glob(rnn_dir+'/*'+session+"*.npz")[0])
b_rnn = data_rnn['b_rnn']
print ("rnn data: ", b_rnn.shape)

# load svm
data_svm = np.load(glob.glob(svm_dir+'/*'+session+"*.npy")[0])
data_svm = data_svm[:data_svm.shape[0]//2]
print ("svm data: ", data_svm.shape)

ax=plt.subplot(111)

# plot rnn
std = np.std(b_rnn,1)
mean = np.mean(b_rnn,1)
t=np.arange(b_rnn.shape[0])-9.5
plt.plot(t, mean, color='black')
ax.fill_between(t, mean-std, mean+std, color='black', alpha = 0.2)

# plot svm
std = np.std(data_svm,1)
mean = np.mean(data_svm,1)
t=np.arange(data_svm.shape[0])/30.-10
plt.plot(t, mean, color='blue')
ax.fill_between(t, mean-std, mean+std, color='blue', alpha = 0.2)

plt.xlim(-10,0)
plt.ylim(0.3, 1.0)
plt.show()


rnn data:  (10, 10)
svm data:  (300, 10)


In [113]:
data = np.load('/media/cat/4TBSSD/yuki/IJ2/concatenation_tests/IJ2pm_Feb29_30Hz_random.npy')
print (data.shape)

(93, 6, 900)


In [178]:
animals = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
root_dir = '/media/cat/4TBSSD/yuki/'
ctr=0
clrs=['black','blue','red','green','magenta','cyan']
for animal in animals:
    fnames = np.load('/media/cat/4TBSSD/yuki/'+animal+'/tif_files.npy')
    n = []
    for fname in fnames:
        session = os.path.split(fname)[1][:-4]
        try:
            data = np.loadtxt(root_dir+animal+'/tif_files/'+session+'/'+session+'_all_locs_selected.txt')
        except:
            continue
        n.append(len(data))

    median = np.mean(n)
    plt.plot([median,median],[0,25],'--',
             linewidth=3,
             c=clrs[ctr])

    #
    total_trials = np.sum(n)
    bins=np.arange(0,210,10)
    y = np.histogram(n, bins)
    plt.plot(y[1][:-1]+5, y[0],
             linewidth=4,
            label=animal+" - "+str(int(median)),
            c=clrs[ctr])
    
    ctr+=1
plt.xlim(bins[0],bins[-1])
plt.legend(fontsize=34)
plt.ylim(0,25)
plt.show()

In [33]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Mar2_30Hzcode_04_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz')
acc = data['accuracy']
print (acc.shape)

(1771, 10)


In [185]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/blue_light_frame_triggers.npz')

frame_triggers = data['img_frame_triggers']
print (frame_triggers)

[ 7353 22587 31547]


In [186]:
start = data['start_blue']
end = data['end_blue']
print (start,end)



77 39207
